In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [6]:
## Load the ANN Trained Model, scaler picker, onehhot
model = load_model('model.h5')

## Load the encoders and scalers
with open('label_encoder_gender.pkl', 'rb') as file:
  label_encoder_gender = pickle.load(file)

with open('one_hot_encoder_geo.pkl', 'rb') as file:
  one_hot_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
  scaler = pickle.load(file)


In [7]:
input_data = {
  'CreditScore' : 600,
  'Geography' : 'France',
  'Gender' : 'Male',
  'Age' : 40,
  'Tenure' : 3,
  'Balance' : 60000,
  'NumOfProducts' : 2,
  'HasCrCard' : 1,
  'IsActiveMember' : 1,
  'EstimatedSalary' : 50000
}

In [8]:
input_df = pd.DataFrame([input_data])

In [9]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [10]:
geo_encoder = one_hot_encoder_geo.transform(input_df[['Geography']])

In [11]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

In [13]:
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [14]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [16]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [17]:
input_scaled = scaler.transform(input_df)

In [18]:
## Prediction

prediction = model.predict(input_scaled)

1/1 [==============================] - 0s 87ms/step


In [19]:
prediction

array([[0.02600233]], dtype=float32)

In [21]:
prediction_proba = prediction[0][0]

In [23]:
prediction_proba

0.026002325

In [24]:
if(prediction_proba > 0.5):
  print('The customer is likely to churn')
else:
  print('The customer is not likely to churn')

The customer is not likely to churn


In [27]:
one_hot_encoder_geo

,categories,'auto'
,drop,None
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


[array(['France', 'Germany', 'Spain'], dtype=object)]

ValueError: Found array with dim 3, while dim <= 2 is required.